In [24]:
import os
import json
import numpy as np
import pandas as pd
import plotly.express as px

In [28]:
df = pd.read_csv('./data/ceidg_data_classif_cleaned.csv')

In [29]:
df.sample(4)

,Unnamed: 0,RandomDate,MonthOfStartingOfTheBusiness,QuarterOfStartingOfTheBusiness,MainAddressVoivodeship,MainAddressCounty,MainAddressTERC,CorrespondenceAddressVoivodeship,CorrespondenceAddressCounty,CorrespondenceAddressTERC,...,ShareholderInOtherCompanies,PKDMainSection,PKDMainDivision,PKDMainGroup,PKDMainClass,NoOfUniquePKDSections,NoOfUniquePKDDivsions,NoOfUniquePKDGroups,NoOfUniquePKDClasses,Target
1459377,1459377,2017-11-15,June,2,ŁÓDZKIE,PIOTRKÓW TRYBUNALSKI,1062011.0,ŁÓDZKIE,PIOTRKÓW TRYBUNALSKI,1062011.0,...,False,C,31.0,310.0,3101.0,7,9,16,25,False
40334,40334,2017-12-08,October,4,MAZOWIECKIE,PIASECZYŃSKI,1418064.0,MAZOWIECKIE,PIASECZYŃSKI,1418064.0,...,False,J,61.0,611.0,6110.0,10,17,41,98,False
443635,443635,2018-09-12,September,3,POMORSKIE,SZTUMSKI,2216055.0,POMORSKIE,SZTUMSKI,2216055.0,...,False,P,85.0,855.0,8559.0,4,4,6,8,True
2165852,2165852,2018-05-06,January,1,ŚLĄSKIE,BIELSKI,2402072.0,ŚLĄSKIE,BIELSKI,2402072.0,...,False,I,56.0,561.0,5610.0,1,1,3,4,False


In [70]:
v_size = df.groupby('MainAddressVoivodeship').size().to_frame('size').reset_index()

In [25]:
with open('./geo/wojewodztwa-min.geojson', 'r', encoding="utf8") as json_file:
    geojson = json.load(json_file)

In [77]:
v_id = pd.DataFrame([v['properties'] for v in geojson['features']])
v_id['nazwa'] = v_id['nazwa'].str.upper()

In [87]:
map_data = pd.merge(v_size, v_id.set_index('nazwa'), left_on='MainAddressVoivodeship', right_index=True)

In [116]:
fig = px.choropleth(map_data, geojson=geojson, color="size", locations="id", featureidkey="properties.id", projection="mercator", color_continuous_scale="peach",
                    labels={'id': 'id województwa', 'size': 'liczba zarejestrowanych działalności'}, hover_name="MainAddressVoivodeship", hover_data=['size'])
fig.update_geos(fitbounds="locations", visible=False, lataxis_range=[50,60], lonaxis_range=[0, 30])
fig.write_html("voivodeships.html")